# **CSC 4008 Assignment 4 (Q2) - 119020062 许文仲**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u362-ga-0ubuntu1~20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/drive/My Drive')

Mounted at /content/gdrive


In [ ]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


from pyspark.sql import SparkSession
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext

In [ ]:
# spark = SparkSession.builder.getOrCreate()
sc = pyspark.SparkContext.getOrCreate()
path_us = "/content/gdrive/MyDrive/Colab Notebooks/CSC 4008/user-shows.txt"
path_s = "/content/gdrive/MyDrive/Colab Notebooks/CSC 4008/shows.txt"

us = sc.textFile(path_us)
show = sc.textFile(path_s)
# rdd.collect()

In [ ]:
# get the first level friend RDD
rdd_us = us.map(lambda x: x.split(" "))


In [ ]:
# find Q with size 9985 * 9985 for users
def find_su(line):
  # 找到每个点属于哪一类
  line = list(line)
  p = list(map(int, line))
  p = np.array(p)
  return np.sum(p)

test = rdd_us.map(find_su)
P_list = test.collect()
P = np.diag(P_list)
P

array([[35,  0,  0, ...,  0,  0,  0],
       [ 0, 26,  0, ...,  0,  0,  0],
       [ 0,  0, 44, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  5,  0,  0],
       [ 0,  0,  0, ...,  0, 30,  0],
       [ 0,  0,  0, ...,  0,  0, 19]])

In [ ]:
# find Q with size 563 * 563 for items
def find_Q(line):
  # 找到每个点属于哪一类
  line = list(line)
  p = list(map(int, line))
  p = np.array(p)
  return p

temp = rdd_us.map(find_Q)
R = np.array(temp.collect())
RT = R.T
Q_list = RT.sum(axis = 1)
Q = np.diag(Q_list)

In [ ]:
P_arr = np.array(P_list)
Q_arr = np.array(Q_list)
sqrt_P = np.sqrt(P_arr)
sqrt_Q = np.sqrt(Q_arr)
invsqrt_P = 1/sqrt_P
invsqrt_Q = 1/sqrt_Q
diainvsqrt_P = np.diag(invsqrt_P)
diainvsqrt_Q = np.diag(invsqrt_Q)

In [ ]:
tempu = np.dot(diainvsqrt_P, R)
tempi = np.dot(diainvsqrt_Q, RT)


In [ ]:
S_u = np.dot(tempu, tempu.T)
S_i = np.dot(tempi, tempi.T)

In [ ]:
Tao_u = np.dot(S_u, R)
Tao_i = np.dot(R, S_i)

In [ ]:
Au = Tao_u[499]
Ai = Tao_i[499]

In [ ]:
uchosen_list = Au[:100]
ichosen_list = Ai[:100]

In [ ]:
ufive = np.array(uchosen_list)
k = 5  #前k个
uindex = ufive.argsort()[-k:][::-1]
uindex

array([96, 74, 45, 60,  9])

In [ ]:
ifive = np.array(ichosen_list)
k = 5  #前k个
iindex = ifive.argsort()[-k:][::-1]
iindex

array([96, 74, 60, 45, 82])

In [ ]:
rdd_s = show.map(lambda x: x.split("\t"))
show_list = rdd_s.collect()
len(show_list)

563

In [ ]:
print("The results is given by U_U")
for i in uindex:
  print(show_list[i])
print("-------------------------")
print("The results is given by I_I")
for i in iindex:
  print(show_list[i])


The results is given by U_U
['"FOX 28 News at 10pm"']
['"Family Guy"']
['"2009 NCAA Basketball Tournament"']
['"NBC 4 at Eleven"']
['"Two and a Half Men"']
-------------------------
The results is given by I_I
['"FOX 28 News at 10pm"']
['"Family Guy"']
['"NBC 4 at Eleven"']
['"2009 NCAA Basketball Tournament"']
['"Access Hollywood"']
